# Классификация и линейная регрессия со scikit-learn

## Часть 2. Алгоритм к-ближайших соседей.

kNN расшифровывается как k Nearest Neighbor или k Ближайших Соседей — это один из самых простых алгоритмов классификации, также иногда используемый в задачах регрессии.

### Описание метода

Описание метода представлено здесь:

* RUS: [Алгоритм k-ближайших соседей // Основы машинного обучения](https://youtu.be/2ufg0G-0RRc)
* RUS: [Алгоритм K-ближайших соседей в Python и Scikit-Learn](https://pythobyte.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn-31cdde4d/)
* RUS: [Алгоритм k-ближайших соседей](http://datascientist.one/k-nearest-neighbors-algorithm/)
* RUS: [Основные метрики задач классификации в машинном обучении](https://webiomed.ai/blog/osnovnye-metriki-zadach-klassifikatsii-v-mashinnom-obuchenii/)
* RUS: [Классификатор kNN](https://habr.com/ru/post/149693/)
* ENG: [Обучение модели машинного обучения с помощью перекрестной проверки](https://docs.microsoft.com/ru-ru/dotnet/machine-learning/how-to-guides/train-machine-learning-model-cross-validation-ml-net)
* ENG: [Разница между K-средним и K-ближайшим соседом](http://www.devcoons.com/difference-k-means-k-nearest-neighbor-algorithm/)
* RUS: [Метод k-ближайших соседей (K-nearest neighbor)](https://wiki.loginom.ru/articles/k-nearest-neighbor.html)

In [ ]:
!pip install kaggle

В ячейке ниже загружаем kaggle.json для вашей среды выполнения Colab, полученный на https://www.kaggle.com/.



In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

### Задание

1. Выберите набор данных (датасет) для решения задачи классификации или регресии.
1. В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.
1. С использованием метода `train_test_split` разделите выборку на обучающую и тестовую.
1. Обучите модель ближайших соседей для произвольно заданного гиперпараметра `K`. Оцените качество модели с помощью трех подходящих для задачи метрик.
1. Постройте модель и оцените качество модели с использованием кросс-валидации. Проведите эксперименты с тремя различными стратегиями кросс-валидации.
1. Произведите подбор гиперпараметра `K` с использованием `GridSearchCV` и кросс-валидации.
1. Повторите пункт 4 для найденного оптимального значения гиперпараметра `K`. Сравните качество полученной модели с качеством модели, полученной в пункте 4.
1. Постройте кривые обучения и валидации.


### Выбор датасета

Для самостоятельной работы предлагается выбрать один из датасетов

* [Glass Classification](https://www.kaggle.com/uciml/glass)

* [Iris Flower Dataset](https://www.kaggle.com/arshid/iris-flower-dataset)

В данном блокноте рассматривается [Classifying wine varieties](https://www.kaggle.com/brynja/wineuci).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.model_selection import KFold, RepeatedKFold, LeaveOneOut, LeavePOut, ShuffleSplit, StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import roc_curve,confusion_matrix, roc_auc_score, accuracy_score, balanced_accuracy_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

plt.style.use('ggplot')

# 'Магическая' функция matplotlib
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')

Получаем датасет:

In [ ]:
!cd /content/
!mkdir wineuci
!cd /content/wineuci

In [ ]:
!kaggle datasets download -d brynja/wineuci -p /content/wineuci

In [ ]:
!unzip /content/wineuci/wineuci.zip -d /content/wineuci

### Загрузка набора данных

In [ ]:
CURENT_DIR = '/content/wineuci'

In [ ]:
# Загрузить набор данных
data = pd.read_csv(CURENT_DIR+'/Wine.csv', delimiter=',', encoding = "ISO-8859-1", 
                     names = ('Class','Alcohol','Malic acid','Ash','Alcalinity of ash',
                                'Magnesium','Total phenols','Flavanoids','Nonflavanoid phenols',
                                'Proanthocyanins','Color intensity','Hue','OD','Proline')) 

# Распечать первые 5 строк фрейма данных
data.head()

### Первичный осмотр выбранного датасета

In [ ]:
# Список колонок с типами данных
data.dtypes

In [ ]:
for col in data.columns:
    print('{} - {}'.format(col, data[data[col].isnull()].shape[0]))

In [ ]:
data.shape 

### Разделяем тестовые и данные для обучения

In [ ]:
X = data.drop('Class',axis=1).values
y = data['Class'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42, stratify=y)
print('X_train: {}  y_train: {}'.format(X_train.shape, y_train.shape))
print('X_test: {}  y_test: {}'.format(X_test.shape, y_test.shape))

### Обучение модели ближайших соседей

**Оптимизация гиперпараметров** — задача машинного обучения по выбору набора оптимальных гиперпараметров для обучающего алгоритма.

Одни и те же виды моделей машинного обучения могут требовать различные предположения, веса или скорости обучения для различных видов данных. Эти параметры называются гиперпараметрами и их следует настраивать так, чтобы модель могла оптимально решить задачу обучения.




In [ ]:
# Настройка массивов для хранения точности обучения и тестирования
neighbors = np.arange(1,14)
len(neighbors) 

### Обучение при различном количестве соседей

In [ ]:
# Вернуть новый массив заданной формы и типа без инициализации записей.
# При первом вызове функции все элементы окажутся случайными:
train_accuracy =np.empty(len(neighbors))
test_accuracy = np.empty(len(neighbors))

for i,k in enumerate(neighbors):
    # Настройка классификатора Knn с K соседями
    knn = KNeighborsClassifier(n_neighbors=k)
    
    # Обучить модель
    knn.fit(X_train, y_train)
    
    # Вычислить точность на тренировочном наборе
    train_accuracy[i] = knn.score(X_train, y_train)
    
    # Вычислить точность на тестовом наборе
    test_accuracy[i] = knn.score(X_test, y_test)

In [ ]:
# Построить набор
plt.title('k-NN Различное количество соседей')
plt.plot(neighbors, train_accuracy, label='Точность на обучающем наборе')
plt.plot(neighbors, test_accuracy, label='Точность на тестовом наборе')
plt.legend()
plt.xlabel('Количество соседей')
plt.ylabel('Точность')
plt.show()

### Изучение работы KNeighborsClassifier



Классификатор, реализующий голосование ближайших соседей.

In [ ]:
# Настройка классификатора knn с K соседями. Указываем искать 10 групп-соседей
knn = KNeighborsClassifier(n_neighbors=10)

In [ ]:
# Обучаем модель
knn.fit(X_train,y_train)

In [ ]:
# Приведет для примера:
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

In [ ]:
# Получить точность. Примечание: в случае алгоритмов классификации метод оценки представляет собой точность.
knn.score(X_test,y_test)

См подробнее об [Precision, recall и F-мера](https://habr.com/ru/company/ods/blog/328372/).

In [ ]:
from sklearn.metrics import classification_report

# precision - точность 
y_pred = knn.predict(X_test)
print(classification_report(y_test,y_pred))

### Точность

In [ ]:
cl1_1 = KNeighborsClassifier(n_neighbors=7)
cl1_1.fit(X_train, y_train)
target1_1 = cl1_1.predict(X_test)
accuracy_score(y_test, target1_1)

In [ ]:
y_pred = knn.predict(X_test)
confusion_matrix(y_test,y_pred)
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

### ROC-кривая



ROC-кривая (Receiver Operator Characteristic) – кривая, которая наиболее часто используется для представления результатов бинарной классификации в машинном обучении. Название пришло из систем обработки сигналов. Поскольку классов два, один из них называется классом с положительными исходами, второй – с отрицательными исходами. ROC-кривая показывает зависимость количества верно классифицированных положительных примеров от количества неверно классифицированных отрицательных примеров. В терминологии ROC-анализа первые называются истинно положительным, вторые – ложно отрицательным множеством. При этом предполагается, что у классификатора имеется некоторый параметр, варьируя который, мы будем получать то или иное разбиение на два класса. Этот параметр часто называют порогом, или точкой отсечения (cut-off value). В зависимости от него будут получаться различные величины ошибок I и II рода.

Подробнее см. [здесь](https://basegroup.ru/community/articles/logistic).

И еще см. [здесь](https://habr.com/ru/post/228963/), тут понятнее.



In [ ]:
y_pred_proba = knn.predict_proba(X_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba, pos_label=2)

plt.plot([0,1],[0,1],'k--')
plt.plot(fpr,tpr, label='Knn')
plt.xlabel('fpr')
plt.ylabel('tpr')
plt.title('Knn(n_neighbors=7) ROC curve')
plt.show()

### Кросс-валидация

[Кросс-валидация](https://neerc.ifmo.ru/wiki/index.php?title=Кросс-валидация)

[Перекрестная проверка K-Fold - Введение в машинное обучение](https://youtu.be/TIgfjmp-4BA)

In [ ]:
param_grid = {'n_neighbors':np.arange(1,14)}
knn = KNeighborsClassifier()
knn_cv= GridSearchCV(knn,param_grid,cv=5)
knn_cv.fit(X_train,y_train)

`GridSearchCV` – это очень мощный инструмент для автоматического подбирания параметров для моделей машинного обучения. GridSearchCV находит наилучшие параметры, путем обычного перебора: он создает модель для каждой возможной комбинации параметров. Важно отметить, что такой подход может быть весьма времязатратным.

Подробнее [здесь](https://vc.ru/ml/147132-kak-avtomaticheski-podobrat-parametry-dlya-modeli-mashinnogo-obucheniya-ispolzuem-gridsearchcv).

In [ ]:
 GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       n_jobs=None,
       param_grid=param_grid,
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [ ]:
knn_cv.best_score_ 

In [ ]:
knn_cv.best_params_ 

### K-fold

[Разновидности скользящего контроля](http://www.machinelearning.ru/wiki/index.php?title=CV)




Данная стратегия работает в соответствии с определением кросс-валидации.

Каждой стратегии в `scikit-learn` ставится в соответствии специальный класс-итератор, который может быть указан в качестве параметра cv функций `cross_val_score` и `cross_validate`.

In [ ]:
scores = cross_val_score(KNeighborsClassifier(n_neighbors=4), 
                         X, y, 
                         cv=KFold(n_splits=5))

# Значение метрики accuracy для 5 фолдов
scores

In [ ]:
# Усредненное значение метрики accuracy для 5 фолдов
np.mean(scores) 

In [ ]:
scoring = {'precision': 'precision_weighted', 
           'recall': 'recall_weighted',
           'f1': 'f1_weighted'}

scores = cross_validate(KNeighborsClassifier(n_neighbors=4), 
                        X, y, scoring=scoring, 
                        cv=KFold(n_splits=5), return_train_score=True)
scores

### Leave One Out (LOO)



В тестовую выборку помещается единственный элемент (One Out). Количество фолдов в этом случае определяется автоматически и равняется количеству элементов.

Данный метод более ресурсоемкий чем KFold.

Существует эмпирическое правило, что вместо Leave One Out лучше использовать KFold на 5 или 10 фолдов.

In [ ]:
loo = LeaveOneOut()
loo.get_n_splits(X)

for train_index, test_index in loo.split(X):
   y_train, y_test = y[train_index], y[test_index]

### Repeated K-Fold


In [ ]:
scores2 = cross_val_score(KNeighborsClassifier(n_neighbors=4), 
                         X, y, 
                         cv=RepeatedKFold(n_splits=5, n_repeats=2))
scores2

### Обучение с оптимальным K

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42, stratify=y)

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train,y_train)
knn.score(X_test,y_test)

### Построение кривых обучения

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):

    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
plot_learning_curve(KNeighborsClassifier(n_neighbors=4), 'n_neighbors=4', 
                    X_train, y_train, cv=5)

### Построение кривой валидации

In [ ]:
def plot_validation_curve(estimator, title, X, y, 
                          param_name, param_range, cv, 
                          scoring="accuracy"):
                                                   
    train_scores, test_scores = validation_curve(
        estimator, X, y, param_name=param_name, param_range=param_range,
        cv=cv, scoring=scoring, n_jobs=1)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.title(title)
    plt.xlabel(param_name)
    plt.ylabel("Score")
    plt.ylim(0.0, 1.1)
    lw = 2
    plt.plot(param_range, train_scores_mean, label="Training score",
                 color="darkorange", lw=lw)
    plt.fill_between(param_range, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.2,
                     color="darkorange", lw=lw)
    plt.plot(param_range, test_scores_mean, label="Cross-validation score",
                 color="navy", lw=lw)
    plt.fill_between(param_range, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.2,
                     color="navy", lw=lw)
    plt.legend(loc="best")
    return plt

In [ ]:
n_range = np.array(range(5,55,5))
plot_validation_curve(KNeighborsClassifier(n_neighbors=4), 'knn', 
                      X_train, y_train, 
                      param_name='n_neighbors', param_range=n_range, 
                      cv=5, scoring="accuracy")